# Cleaning Scrapping Data

In [1]:
base_path = '../data_public/sitemap_dumps/'

# Import des librairies
import pandas as pd
import pyarrow
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# créer deux listes contenant les noms des fichiers scrappés
tv_sources = os.listdir(base_path+'media_type=tv')
webpress_sources = os.listdir(base_path+'media_type=webpress')
filelist_tv = []
for source in tv_sources:
    for root, dirs, files in os.walk(base_path+'media_type=tv/'+source):
	    for file in files:
            #append the file name to the list
		    filelist_tv.append(os.path.join(root,file))

filelist_webpress = []
for source in webpress_sources:
    for root, dirs, files in os.walk(base_path+'media_type=webpress/'+source):
        for file in files:
            #append the file name to the list
            filelist_webpress.append(os.path.join(root,file))

In [2]:
#  Importer les fichiers parquet dans un dataframe pandas
df_tv = pd.DataFrame()
for file in filelist_tv:
    df_tv = df_tv.append(pd.read_parquet(file, engine='pyarrow'))

df_webpress = pd.DataFrame()
for file in filelist_webpress:
    df_webpress = df_webpress.append(pd.read_parquet(file, engine='pyarrow'))

In [3]:
# Concaténer les deux dataframes

df = pd.concat([df_tv, df_webpress], axis=0)

In [4]:
# vérifier les pourcentages de valeurs manquantes par colonnes
print('Pourcentage de valeurs manquantes par colonnes pour le dataframe : ')
print(df.isnull().sum()/df.shape[0]*100)

Pourcentage de valeurs manquantes par colonnes pour le dataframe : 
url                       0.000000
publication_name          0.000000
publication_language      0.000000
news_publication_date     0.000000
news_title                0.000000
image_loc                 1.694514
image_caption             8.705107
sitemap                   0.000000
sitemap_size_mb           0.000000
download_date             0.000000
media                     0.000000
section                   0.000000
media_type               75.983981
news                     56.982603
news_publication         56.982603
image                    57.490742
news_keywords            65.608044
etag                     75.669624
sitemap_last_modified    54.207217
lastmod                  57.908449
news_genres              99.829903
news_access              99.608130
changefreq               95.047800
priority                 95.047800
dtype: float64


In [5]:
# Supprimer les colonnes avec plus de 50% de valeurs manquantes

df = df.dropna(axis=1, thresh=df.shape[0]*0.5)

In [6]:
# Remove all dates matching the pattern `\d{2}\/\d{2}` in news_title
import re

df['news_title'] = df['news_title'].apply(lambda x: re.sub(r'\d{2}\/\d{2}', '', x))

In [7]:
# check columns dtype

df.dtypes

url                              object
publication_name                 object
publication_language             object
news_publication_date    datetime64[ns]
news_title                       object
image_loc                        object
image_caption                    object
sitemap                          object
sitemap_size_mb                 float64
download_date            datetime64[ns]
media                            object
section                          object
dtype: object

In [8]:
# convertir la colonne section de type array en type list

df['section'] = df['section'].apply(lambda x: x.tolist())

In [9]:
#print the first value of the column section

print(df['section'].iloc[0])

['tech', 'bons-plans']


In [10]:
# Chercher les bulletins météos dans la colonne section

df['is_weather'] = df['section'].apply(lambda x: 'meteo' in x)



print('Pourcentage de bulletins météos dans le dataframe : ')
print(df['is_weather'].sum()/df.shape[0]*100)

# Extraire les bulletins météos dans un dataframe séparé

df_weather = df[df['is_weather'] == True]

Pourcentage de bulletins météos dans le dataframe : 
1.2057531651020585


In [11]:
# Chercher les articles ayant 'JT' dans le titre

df['is_jt'] = df['news_title'].apply(lambda x: 'JT' in x)

print('Pourcentage d\'articles ayant "JT" dans le titre : ')
print(df['is_jt'].sum()/df.shape[0]*100)

# Extraire les articles ayant 'JT' dans le titre dans un dataframe séparé

df_jt = df[df['is_jt'] == True]

Pourcentage d'articles ayant "JT" dans le titre : 
0.08827835673068642


In [12]:
# Chercher les replay-emissions dans la colonne section

df['is_replay'] = df['section'].apply(lambda x: 'replay-emissions' in x)



print('Pourcentage de is_replay dans le dataframe : ')
print(df['is_replay'].sum()/df.shape[0]*100)

# Extraire les bulletins météos dans un dataframe séparé

df_replay = df[df['is_replay'] == True]

Pourcentage de is_replay dans le dataframe : 
12.259925932305572


In [17]:
# Chercher les replay-emissions dans la colonne section

df['is_bonsoir'] = df['section'].apply(lambda x: 'bonsoir-lyon' in x)



print('Pourcentage de is_bonsoir dans le dataframe : ')
print(df['is_bonsoir'].sum()/df.shape[0]*100)

# Extraire les bulletins météos dans un dataframe séparé

df_bonsoir = df[df['is_bonsoir'] == True]

Pourcentage de is_bonsoir dans le dataframe : 
0.4823012660408234
